In [1]:
!pip install trimesh
!pip install mcubes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 12.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 6.7 MB/s eta 0:00:00:00:01


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import mcubes
from tqdm import tqdm
from skimage import measure
import trimesh

# ---------------- CONFIG ----------------
CHECKPOINT_PATH = "/kaggle/input/model-3k/pytorch/default/1/ckpt_03000.pth"
OUTPUT_MESH = "nerf_mesh2.ply"

GRID_RES = 384    # 128 = fast, 256 = good, 384 = heavy
SIGMA_THRESHOLD = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

L_POS = 10
L_DIR = 4
# ---------------------------------------


# ------------ POSITIONAL ENCODING ------------
def posenc(x, L):
    out = [x]
    for i in range(L):
        out.append(torch.sin((2 ** i) * torch.pi * x))
        out.append(torch.cos((2 ** i) * torch.pi * x))
    return torch.cat(out, dim=-1)


# ------------ NERF MODEL (UNCHANGED) ------------
class NeRF(nn.Module):
    def __init__(self, W=256):
        super().__init__()
        self.fc = nn.ModuleList(
            [nn.Linear(63, W)] + [nn.Linear(W, W) for _ in range(7)]
        )
        self.sigma = nn.Linear(W, 1)
        self.rgb = nn.Sequential(
            nn.Linear(W + 27, W),
            nn.ReLU(),
            nn.Linear(W, 3)
        )

    def forward(self, x):
        pts, views = torch.split(x, [63, 27], -1)
        h = pts
        for l in self.fc:
            h = torch.relu(l(h))
        sigma = torch.relu(self.sigma(h))
        rgb = torch.sigmoid(self.rgb(torch.cat([h, views], -1)))
        return rgb, sigma


# ------------ LOAD MODEL ------------
model = NeRF().to(DEVICE)
ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)

if isinstance(ckpt, dict) and "model_f" in ckpt:
    model.load_state_dict(ckpt["model_f"])
else:
    model.load_state_dict(ckpt)

model.eval()
print("Model loaded")


# ------------ CREATE 3D GRID ------------
lin = torch.linspace(-0.6, 0.6, GRID_RES)
X, Y, Z = torch.meshgrid(lin, lin, lin, indexing="ij")
pts = torch.stack([X, Y, Z], dim=-1).reshape(-1, 3).to(DEVICE)

# Dummy view direction (not used for density)
views = torch.zeros_like(pts)

# ------------ QUERY SIGMA FIELD ------------
sigmas = []

CHUNK = 65536
with torch.no_grad():
    for i in tqdm(range(0, pts.shape[0], CHUNK)):
        p = pts[i:i+CHUNK]
        v = views[i:i+CHUNK]

        p_enc = posenc(p, L_POS)
        v_enc = posenc(v, L_DIR)

        _, sigma = model(torch.cat([p_enc, v_enc], -1))
        sigmas.append(sigma.squeeze(-1).cpu())

sigma_grid = torch.cat(sigmas).reshape(GRID_RES, GRID_RES, GRID_RES).numpy()

print("Density grid computed")

from scipy.ndimage import gaussian_filter

# Smooth density field
sigma_grid = gaussian_filter(sigma_grid, sigma=0.5)


# marching cubes
verts, faces, normals, values = measure.marching_cubes(
    sigma_grid,
    level=SIGMA_THRESHOLD
)

# normalize verts to world coordinates
verts = verts / (GRID_RES - 1) * 2.4 - 1.2

mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals)
mesh.export(OUTPUT_MESH)

print(f"Mesh saved to {OUTPUT_MESH}")

✅ Model loaded


100%|██████████| 864/864 [00:27<00:00, 31.78it/s]


✅ Density grid computed
✅ Mesh saved to nerf_mesh2.ply
